https://adventofcode.com/2021/day/6

<article class="day-desc"><h2>--- Day 6: Lanternfish ---</h2><p>The sea floor is getting steeper. Maybe the sleigh keys got carried this way?</p>
<p>A massive school of glowing <a href="https://en.wikipedia.org/wiki/Lanternfish" target="_blank">lanternfish</a> swims past. They must spawn quickly to reach such large numbers - maybe <em>exponentially</em> quickly? You should model their growth rate to be sure.</p>
<p>Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, <span title="I heard you like lanternfish.">each lanternfish creates a new lanternfish</span> once every <em>7</em> days.</p>
<p>However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents <em>the number of days until it creates a new lanternfish</em>.</p>
<p>Furthermore, you reason, a <em>new</em> lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.</p>
<p>So, suppose you have a lanternfish with an internal timer value of <code>3</code>:</p>
<ul>
<li>After one day, its internal timer would become <code>2</code>.</li>
<li>After another day, its internal timer would become <code>1</code>.</li>
<li>After another day, its internal timer would become <code>0</code>.</li>
<li>After another day, its internal timer would reset to <code>6</code>, and it would create a <em>new</em> lanternfish with an internal timer of <code>8</code>.</li>
<li>After another day, the first lanternfish would have an internal timer of <code>5</code>, and the second lanternfish would have an internal timer of <code>7</code>.</li>
</ul>
<p>A lanternfish that creates a new fish resets its timer to <code>6</code>, <em>not <code>7</code></em> (because <code>0</code> is included as a valid timer value). The new lanternfish starts with an internal timer of <code>8</code> and does not start counting down until the next day.</p>
<p>Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:</p>
<pre><code>3,4,3,1,2</code></pre>
<p>This list means that the first fish has an internal timer of <code>3</code>, the second fish has an internal timer of <code>4</code>, and so on until the fifth fish, which has an internal timer of <code>2</code>. Simulating these fish over several days would proceed as follows:</p>
<pre><code>Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
</code></pre>
<p>Each day, a <code>0</code> becomes a <code>6</code> and adds a new <code>8</code> to the end of the list, while each other number decreases by 1 if it was present at the start of the day.</p>
<p>In this example, after 18 days, there are a total of <code>26</code> fish. After 80 days, there would be a total of <code><em>5934</em></code>.</p>
<p>Find a way to simulate lanternfish. <em>How many lanternfish would there be after 80 days?</em></p>
</article>

In [218]:
import pandas as pd
import numpy as np

In [219]:
f = open("Lanternfish.txt", "r")
raw = f.read()
raw

'1,1,1,1,2,1,1,4,1,4,3,1,1,1,1,1,1,1,1,4,1,3,1,1,1,5,1,3,1,4,1,2,1,1,5,1,1,1,1,1,1,1,1,1,1,3,4,1,5,1,1,1,1,1,1,1,1,1,3,1,4,1,1,1,1,3,5,1,1,2,1,1,1,1,4,4,1,1,1,4,1,1,4,2,4,4,5,1,1,1,1,2,3,1,1,4,1,5,1,1,1,3,1,1,1,1,5,5,1,2,2,2,2,1,1,2,1,1,1,1,1,3,1,1,1,2,3,1,5,1,1,1,2,2,1,1,1,1,1,3,2,1,1,1,4,3,1,1,4,1,5,4,1,4,1,1,1,1,1,1,1,1,1,1,2,2,4,5,1,1,1,1,5,4,1,3,1,1,1,1,4,3,3,3,1,2,3,1,1,1,1,1,1,1,1,2,1,1,1,5,1,3,1,4,3,1,3,1,5,1,1,1,1,3,1,5,1,2,4,1,1,4,1,4,4,2,1,2,1,3,3,1,4,4,1,1,3,4,1,1,1,2,5,2,5,1,1,1,4,1,1,1,1,1,1,3,1,5,1,2,1,1,1,1,1,4,4,1,1,1,5,1,1,5,1,2,1,5,1,1,1,1,1,1,1,1,1,1,1,1,3,2,4,1,1,2,1,1,3,2\n'

In [220]:
peces_iniciales = np.fromstring(raw, sep = ',', dtype= int)
peces_iniciales

array([1, 1, 1, 1, 2, 1, 1, 4, 1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 3,
       1, 1, 1, 5, 1, 3, 1, 4, 1, 2, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 4, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4, 1, 1, 1, 1, 3,
       5, 1, 1, 2, 1, 1, 1, 1, 4, 4, 1, 1, 1, 4, 1, 1, 4, 2, 4, 4, 5, 1,
       1, 1, 1, 2, 3, 1, 1, 4, 1, 5, 1, 1, 1, 3, 1, 1, 1, 1, 5, 5, 1, 2,
       2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 5, 1, 1, 1,
       2, 2, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 4, 3, 1, 1, 4, 1, 5, 4, 1, 4,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 4, 5, 1, 1, 1, 1, 5, 4, 1, 3,
       1, 1, 1, 1, 4, 3, 3, 3, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 5, 1, 3, 1, 4, 3, 1, 3, 1, 5, 1, 1, 1, 1, 3, 1, 5, 1, 2, 4, 1,
       1, 4, 1, 4, 4, 2, 1, 2, 1, 3, 3, 1, 4, 4, 1, 1, 3, 4, 1, 1, 1, 2,
       5, 2, 5, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 3, 1, 5, 1, 2, 1, 1, 1, 1,
       1, 4, 4, 1, 1, 1, 5, 1, 1, 5, 1, 2, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 2, 4, 1, 1, 2, 1, 1, 3, 2])

In [66]:
peces_actuales = np.copy(peces_iniciales)
for dia in range(256):
    if dia>0:
        idx_peces_0 = np.where(peces_actuales == 0)
        if idx_peces_0[0].size>0:
            peces_actuales[idx_peces_0] = 7
            peces_actuales=np.append(peces_actuales,
                                     np.full(idx_peces_0[0].size, 9, dtype=int))
    peces_actuales-=1
    #print(dia+1, peces_actuales)
peces_actuales.size

KeyboardInterrupt: 

In [53]:
#13:29
idx_peces_0 = np.where(peces_actuales == 0)
idx_peces_0[0].size

0

In [11]:
peces_iniciales-=1

In [12]:
peces_iniciales

array([ 0,  1,  0, -2, -1])

In [44]:
np.where(peces_actuales == 0)

(array([], dtype=int64),)

In [14]:
peces_iniciales

array([ 0,  1,  0, -2, -1])

In [18]:
peces_iniciales[np.where(peces_iniciales == 3)] = 99

In [19]:
peces_iniciales

array([99,  4, 99,  1,  2])

In [20]:
np.full(5, 8, dtype=int)

array([8, 8, 8, 8, 8])

In [67]:
from sklearn.neural_network import MLPClassifier

In [114]:
peces_iniciales.size

5

In [69]:
l = [3,4,3,1,2]

In [74]:
t = 24
t

24

In [ ]:
80/8

In [86]:
q = 5
for i in range(0,10):
    q=q*2
    print(q)
q

10
20
40
80
160
320
640
1280
2560
5120


5120

In [87]:
5*10*2

100

In [98]:
unique, counts = np.unique(peces_iniciales, return_counts=True)
registro = dict(zip(unique, counts))
registro

{1: 1, 2: 1, 3: 2, 4: 1}

In [ ]:
peces_actuales = np.copy(peces_iniciales)
for dia in range(256):
    if dia>0:
        idx_peces_0 = np.where(peces_actuales == 0)
        if idx_peces_0[0].size>0:
            peces_actuales[idx_peces_0] = 7
            peces_actuales=np.append(peces_actuales,
                                     np.full(idx_peces_0[0].size, 9, dtype=int))
    peces_actuales-=1
    #print(dia+1, peces_actuales)
peces_actuales.size

In [116]:
registro = { x:0 for x in range(9)}
registro

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}

In [117]:
for i in peces_iniciales:
    registro[i]+=1
registro

{0: 0, 1: 1, 2: 1, 3: 2, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0}

In [113]:
for i in range(18):
    

array([4, 4, 4, ..., 8, 8, 8])

In [118]:
np.array(registro)

array({0: 0, 1: 1, 2: 1, 3: 2, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0}, dtype=object)

In [210]:
registro = np.zeros(9).astype(int)

In [211]:
for i in peces_iniciales:
    registro[i]+=1
registro

array([  0, 192,  28,  27,  32,  21,   0,   0,   0])

In [129]:
registro[1:]

array([1, 1, 2, 1, 0, 0, 0, 0])

In [136]:
registro = np.append(registro[1:], registro[0])
registro[5] = registro[0]
registro, registro.sum()

(array([1, 2, 1, 0, 1, 1, 1, 0, 1]), 8)

In [212]:
registro = np.zeros(9).astype(np.int64)
for i in peces_iniciales:
    registro[i]+=1
for i in range(256):
    primero = registro[0]
    registro = np.append(registro[1:], registro[0])
    registro[6]+= primero
    #print(registro, registro.sum())
#registro, registro.sum()
registro.sum()

1732821262171

In [202]:
registro1 = np.zeros(9).astype(int)
for i in [5,6,5,3,4,5,6,7,7,8]:
    registro1[i]+=1
registro1

array([0, 0, 0, 1, 1, 3, 2, 2, 1])

In [203]:
print(registro)
print(registro1)

[0 0 0 1 1 3 2 2 1]
[0 0 0 1 1 3 2 2 1]
